In [1]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
import pandas as pd
import tensorflow as tf

from imbd.trainers import ModelTrainer
from imbd.data import DataLoader
from imbd.preprocessors import DataPreprocessor
from imbd.models import KerasModel

In [2]:
loader = DataLoader()
prepro = DataPreprocessor()


In [3]:
    param_grid = {
        "prepro__variance_selector__threshold": [0.0, 0.01],
        "model__epochs": [10, 20],
        "model__dropout_rate": [0.3, 0.1],
        # "model__estimator__n_estimators": [1000],
        # "model__estimator__max_depth": [5, 10],
        # "model__estimator__alpha": [0, 0.1, 0.01],
        # "model__estimator__lambda": [1, 0.5, 0.1],
        # "model__estimator__subsample": [1, 0.5],
        # "model__estimator__gamma": [0, 2, 10],
    }

In [4]:
# initialization
loader = DataLoader()
preprocessor = DataPreprocessor()
df = loader.build()

# get feature & label
train_features = df.drop(loader.labels, axis=1)
train_labels = df[loader.labels]


In [5]:
base_model = MultiOutputRegressor(XGBRegressor())
# base_nn_model = KerasRegressor(build_fn=create_model, epochs=100)
# base_nn_model = KerasRegressor(build_fn=classifier, epochs=100)
base_nn_model = KerasModel()

In [6]:
# build pipeline
steps = [('prepro', preprocessor), ('model', base_nn_model)]
pipe = Pipeline(steps=steps)

In [7]:
pipe.get_params()

{'memory': None,
 'steps': [('prepro', DataPreprocessor()),
  ('model', <imbd.models.KerasModel at 0x1423b2780>)],
 'verbose': False,
 'prepro': DataPreprocessor(),
 'model': <imbd.models.KerasModel at 0x1423b2780>,
 'prepro__drop_na_by_threshold': NADropper(),
 'prepro__quantization': <imbd.preprocessors.QuantizationTransformer at 0x142343f28>,
 'prepro__fill_na': <imbd.preprocessors.FillNATransformer at 0x142343ef0>,
 'prepro__variance_selector': VarianceFeatureSelector(),
 'prepro__outlier_detection': <imbd.preprocessors.OutlierDetector at 0x142343e80>,
 'prepro__drop_na_by_threshold__na_threshold': 10,
 'prepro__variance_selector__threshold': 0.0,
 'model__build_fn': <function imbd.models.KerasModel.create_func_model(n_features, dropout_rate=0.3)>}

In [8]:
# training
trainer = ModelTrainer(pipe=pipe, param_grid=param_grid, verbose=2)
fitted = trainer.train(train_features, train_labels)

s 2ms/step - loss: 0.0287
Epoch 70/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0289
Epoch 71/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0287
Epoch 72/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0289
Epoch 73/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0289
Epoch 74/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0288
Epoch 75/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0284
Epoch 76/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0286
Epoch 77/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0290
Epoch 78/100
8/8 [==============================] - 0s 5ms/step - loss: 0.0286
Epoch 79/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0288
Epoch 80/100
8/8 [==============================] - 0s 2ms/step - loss: 0.0286
Epoch 81/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0285
Epoch 82/100
8/8 [========

In [9]:
fitted.best_score_

0.20621413610369368

In [10]:
pd.DataFrame(fitted.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__dropout_rate,param_model__epochs,param_prepro__variance_selector__threshold,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.355651,0.284143,0.287637,0.051699,0.3,10,0,"{'model__dropout_rate': 0.3, 'model__epochs': ...",0.227192,0.236089,0.152075,0.205119,0.037683,2
1,1.060265,0.033802,0.270902,0.020839,0.3,10,0.01,"{'model__dropout_rate': 0.3, 'model__epochs': ...",0.232526,0.234802,0.151314,0.206214,0.038831,1
2,4.050157,0.834316,0.547688,0.133316,0.3,100,0,"{'model__dropout_rate': 0.3, 'model__epochs': ...",0.201107,0.206302,0.123238,0.176882,0.037992,7
3,4.467174,0.322863,0.658920,0.086671,0.3,100,0.01,"{'model__dropout_rate': 0.3, 'model__epochs': ...",0.201118,0.207784,0.123434,0.177445,0.038289,5
4,2.100579,0.316068,0.647453,0.208571,0.1,10,0,"{'model__dropout_rate': 0.1, 'model__epochs': ...",0.223066,0.227190,0.147842,0.199366,0.036472,4
5,1.347657,0.013287,0.448769,0.024791,0.1,10,0.01,"{'model__dropout_rate': 0.1, 'model__epochs': ...",0.228172,0.240943,0.144192,0.204436,0.042916,3
6,4.507659,0.209913,0.822383,0.246700,0.1,100,0,"{'model__dropout_rate': 0.1, 'model__epochs': ...",0.200676,0.205851,0.124162,0.176896,0.037349,6
7,4.256617,0.083006,0.679731,0.029866,0.1,100,0.01,"{'model__dropout_rate': 0.1, 'model__epochs': ...",0.200204,0.205056,0.125332,0.176864,0.036493,8


In [11]:
np.abs(fitted.predict(train_features) - train_labels)

,Input_A6_024,Input_A3_016,Input_C_013,Input_A2_016,Input_A3_017,Input_C_050,Input_A6_001,Input_C_096,Input_A3_018,Input_A6_019,Input_A1_020,Input_A6_011,Input_A3_015,Input_C_046,Input_C_049,Input_A2_024,Input_C_058,Input_C_057,Input_A3_013,Input_A2_017
0,0.004014,0.043178,0.003052,0.010233,0.01365,0.049748,0.14488,0.012559,0.012205,0.013137,0.059031,0.019009,0.021889,0.025638,0.001047,0.004156,0.007833,0.008508,0.020316,0.030284
1,0.004014,0.033178,0.002052,0.000233,0.00365,0.049748,0.01512,0.032559,0.002205,0.003137,0.040969,0.021009,0.038111,0.025638,0.000947,0.004156,0.006833,0.014508,0.020316,0.020284
2,0.015986,0.063178,0.003052,0.019767,0.04365,0.049748,0.01512,0.012559,0.022205,0.026863,0.059031,0.020009,0.011889,0.025438,0.000847,0.024156,0.002833,0.015508,0.020316,0.009716
3,0.015986,0.063178,0.002052,0.019767,0.00365,0.051748,0.01512,0.032559,0.017795,0.013137,0.840969,0.019009,0.041889,0.025138,0.001347,0.034156,0.002167,0.013508,0.020316,0.020284
4,0.004014,0.063178,0.003052,0.009767,0.01365,0.051748,0.01512,0.022559,0.012205,0.003137,0.159031,0.020009,0.001889,0.025538,0.000847,0.024156,0.006833,0.017508,0.022316,0.000284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,0.004014,0.033178,0.003052,0.020233,0.01365,0.046048,0.01512,0.022559,0.017795,0.003137,0.059031,0.023009,0.038111,0.025738,0.000847,0.024156,0.011333,0.000508,0.020316,0.020284
335,0.015986,0.023178,0.003052,0.010233,0.01365,0.045948,0.01512,0.022559,0.017795,0.003137,0.059031,0.025009,0.038111,0.026038,0.000447,0.034156,0.011533,0.000808,0.020316,0.020284
336,0.035986,0.033178,0.003052,0.010233,0.02365,0.045748,0.01512,0.022559,0.007795,0.003137,0.059031,0.025009,0.028111,0.025138,0.000747,0.024156,0.011133,0.000092,0.022316,0.030284
337,0.015986,0.033178,0.003052,0.020233,0.00365,0.046148,0.01512,0.022559,0.002205,0.013137,0.040969,0.025009,0.038111,0.025838,0.000547,0.034156,0.012033,0.000608,0.020316,0.020284


In [12]:
pipe.get_params()['model'].get_params()

{'build_fn': <function imbd.models.KerasModel.create_func_model(n_features, dropout_rate=0.3)>}